This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [ ]:
# We installed the necessary packages for this project
!pip install tensorflow opencv-python matplotlib numpy pandas

In [ ]:
!pip install keras

In [ ]:
# I installed kagglehub to download and manage Kaggle datasets easily
!pip install kagglehub

In [ ]:
import kagglehub

# We downloaded the Muffin vs Chihuahua dataset from Kaggle
path = kagglehub.dataset_download("samuelcortinhas/muffin-vs-chihuahua-image-classification")

print("Dataset downloaded to:", path)

Using Colab cache for faster access to the 'muffin-vs-chihuahua-image-classification' dataset.
Dataset downloaded to: /kaggle/input/muffin-vs-chihuahua-image-classification


In [ ]:
import os
print(os.listdir("/kaggle/input/muffin-vs-chihuahua-image-classification"))

['test', 'train']


In [ ]:
train_dir = os.path.join(path, "train")
test_dir  = os.path.join(path, "test")

print("Train folder:", train_dir)
print("Test folder:", test_dir)

Train folder: /kaggle/input/muffin-vs-chihuahua-image-classification/train
Test folder: /kaggle/input/muffin-vs-chihuahua-image-classification/test


In [ ]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

In [ ]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = path + "/train"
test_dir  = path + "/test"

In [ ]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [ ]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [ ]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting and L2 regularization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu',
                  kernel_regularizer=tf.keras.regularizers.l2(0.001),
                  input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu',
                  kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu',
                  kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu',
                 kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# TRAINING THE CNN

early_stop = EarlyStopping(
    monitor='val_loss',  # watch the validation loss
    patience=3,          # stop if no improvement for 3 epochs
    restore_best_weights=True  # restore weights from the best epoch
)
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

Epoch 1/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 0.5726 - loss: 1.0772 - val_accuracy: 0.6899 - val_loss: 0.7575
Epoch 2/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.7205 - loss: 0.7012 - val_accuracy: 0.7831 - val_loss: 0.5858
Epoch 3/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.7847 - loss: 0.5742 - val_accuracy: 0.7979 - val_loss: 0.5628
Epoch 4/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.8045 - loss: 0.5141 - val_accuracy: 0.8434 - val_loss: 0.4464
Epoch 5/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.8045 - loss: 0.4872 - val_accuracy: 0.8212 - val_loss: 0.4469
Epoch 6/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.8286 - loss: 0.4678 - val_accuracy: 0.8794 - val_loss: 0.3803
Epoch 7/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.8402 - loss: 0.4395 - val_accuracy: 0.7884 - val_loss: 0.4989
Epoch 8/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.8304 - loss: 0.4461 - val_accu

In [ ]:
# a. (5 points) Accuracy
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 15s 406ms/step - accuracy: 0.9329 - loss: 0.2393
Test Accuracy: 0.8589527010917664


In [ ]:
# c. (10 points) Rename the trained CNN model to “exercise_6_trained_model_improved.h5”
# SAVE THE MODEL
model.save("exercise_6_trained_model_improved.h5")
print("Model saved as 'exercise_6_trained_model_improved.h5'")

In [ ]:
# b. (10 points) Run 1 and 2’s prediction and confidence
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # pred is P(Muffin) based on flow_from_directory's alphabetical order
    prob_muffin = model.predict(img_array)[0,0]

    # Determine the predicted class and its confidence
    if prob_muffin >= 0.5:
        label = "Muffin"
        confidence = prob_muffin
    else:
        label = "Chihuahua"
        # P(Chihuahua) = 1 - P(Muffin)
        confidence = 1.0 - prob_muffin

    print(f"Prediction: {label} (confidence: {confidence:.2f})")


In [ ]:
# Example usage:
predict_image("run_1.jpg")
predict_image("run_2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Prediction: Muffin (confidence: 0.53)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Prediction: Muffin (confidence: 1.00)
